<a href="https://colab.research.google.com/github/ganeshsprofessional/sdc-GenAI/blob/main/resume_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AI Resume Analyzer in Google Colab

# Install required packages
!pip install openai PyPDF2 langchain --quiet

import openai
import PyPDF2
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Set your OpenAI API key
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your API key

# Function to extract text from PDF resume
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Upload resume
from google.colab import files
uploaded = files.upload()
resume_path = next(iter(uploaded))

# (Optional) Enter job description for matching
job_description = input("Paste the job description (or leave blank to skip matching): ")

# Extract resume text
resume_text = extract_text_from_pdf(resume_path)

# Create LangChain LLM
llm = OpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

# Build prompt for analysis
if job_description.strip():
    prompt = PromptTemplate(
        input_variables=["resume", "job"],
        template=(
            "Analyze the following resume for strengths, weaknesses, and suggestions for improvement. "
            "Then, compare it to the provided job description and give a matching score (0-100), "
            "with a brief explanation of the match.\n\n"
            "Resume:\n{resume}\n\nJob Description:\n{job}"
        )
    )
    final_prompt = prompt.format(resume=resume_text, job=job_description)
else:
    prompt = PromptTemplate(
        input_variables=["resume"],
        template=(
            "Analyze the following resume. Summarize the candidate's strengths, weaknesses, "
            "and provide suggestions for improvement.\n\nResume:\n{resume}"
        )
    )
    final_prompt = prompt.format(resume=resume_text)

# Get analysis from LLM
analysis = llm(final_prompt)

print("\nAI Resume Analysis:\n")
print(analysis)
